In [ ]:
import pandas as pd

# Load the dataset
path = "./9.xlsx"
df = pd.read_excel(path)
data = pd.read_csv('/content/results_Ml.csv')
# Calculate offensive and defensive scores
df['Offensive Score'] = (data['ORtg'])

df['Defensive Score'] = (data['DRtg'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/results_Ml.csv'

In [ ]:
df.to_csv("9_with_scores.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random

# Load player ratings and match data
results_df = pd.read_csv('results_final.csv')  # Player ratings (ORtg, DRtg)
df = pd.read_excel('9.xlsx')  # Original match data



In [ ]:
results_df.sort_values(by='ORtg', ascending=False)

,Unnamed: 0.1,Unnamed: 0,Athlete,ORtg,DRtg,Matches Played
1119,1119,1119,Imani Lewis,280.213535,-145.393547,1
2508,2508,2508,Sadie Stetson,270.374001,75.233206,1
1251,1251,1251,Janae Kent,218.023800,67.122006,1
2446,2446,2446,Rayah Marshall,215.157803,48.298301,1
2615,2615,2615,Sharon Goodman,191.791165,55.605368,1
...,...,...,...,...,...,...
1088,1088,1088,Hannah Simmons,0.000000,91.228397,1
1125,1125,1125,India Otto,0.000000,46.211018,2
2134,2134,2134,Melissa Mwanza,0.000000,75.757656,1
2139,2139,2139,Mercy Ademusayo,0.000000,29.062348,1


In [ ]:
results_df = results_df.rename(columns={'Player': 'Athlete'})

# Merge player ratings into the dataset
def merge_player_ratings(df, results_df):
    df = df.merge(results_df[['Athlete', 'ORtg', 'DRtg']],
                  on='Athlete', how='left')
    return df

df = merge_player_ratings(df, results_df)
df_filtered = df[df['MIN'] >= 5].copy()  # Filter players with significant playtime
df_filtered['Win'] = df_filtered['Win'].astype(int)

In [ ]:
# Load player ratings and match data
results_df = pd.read_csv('results_final.csv')  # Player ratings (ORtg, DRtg)
df = pd.read_excel('9.xlsx')  # Original match data

# Rename 'Player' to 'Athlete' in results_df for consistency
results_df = results_df.rename(columns={'Player': 'Athlete'})

# Merge player ratings into the dataset
def merge_player_ratings(df, results_df):
    df = df.merge(results_df[['Athlete', 'ORtg', 'DRtg']],
                  on='Athlete', how='left')
    return df

df = merge_player_ratings(df, results_df)
df_filtered = df[df['MIN'] >= 5].copy()  # Filter players with significant playtime
df_filtered['Win'] = df_filtered['Win'].astype(int)

# Train a Logistic Regression model to predict win probability
def train_win_predictor(df_filtered):
    player_counts = df_filtered.groupby(['Date', 'Team Display Name'])['Athlete'].count().reset_index()
    player_counts.rename(columns={'Athlete': 'Num_Players'}, inplace=True)

    team_data = df_filtered.groupby(['Date', 'Team Display Name']).agg(
        Team_Offensive_Score=('ORtg', 'sum'),
        Team_Defensive_Score=('DRtg', 'sum'),
        Team_Game_Score=('Game Score', 'sum'),
        Win=('Win', 'first')
    ).reset_index()

    team_data = team_data.merge(player_counts, on=['Date', 'Team Display Name'])
    team_data['Team_Offensive_Score'] /= team_data['Num_Players']
    team_data['Team_Defensive_Score'] /= team_data['Num_Players']
    team_data['Team_Game_Score'] /= team_data['Num_Players']

    df_opponents = df_filtered[['Date', 'Team Display Name', 'Opponet']].drop_duplicates()
    team_df = team_data.merge(df_opponents, on=['Date', 'Team Display Name'])
    team_df = team_df.merge(team_data, left_on=['Date', 'Opponet'],
                            right_on=['Date', 'Team Display Name'], suffixes=('', '_Opp'))

    X = team_df[['Team_Offensive_Score', 'Team_Defensive_Score',
                 'Team_Offensive_Score_Opp', 'Team_Defensive_Score_Opp',]].copy()
    y = team_df['Win'].copy()

    # Handle infinite and NaN values
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    mask = X.notna().all(axis=1)  # Keep only rows where all features are non-NaN
    X = X[mask]
    y = y[mask]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Logistic Regression Model Accuracy: {accuracy:.4f}")

    return model

# Q-Learning Environment
class LineupEnv:
    def __init__(self, team_players, opp_lineup, win_predictor):
        self.team_players = team_players  # List of player names in the team
        self.opp_lineup = opp_lineup      # Fixed opposition lineup (list of 5 player names)
        self.win_predictor = win_predictor
        self.max_lineup_size = 5
        self.current_lineup = []
        self.action_space = len(team_players)  # Actions are indices of players to add/remove
        self.feature_names = ['Team_Offensive_Score', 'Team_Defensive_Score',
                              'Team_Offensive_Score_Opp', 'Team_Defensive_Score_Opp']

    def reset(self):
        self.current_lineup = []
        return tuple(self.current_lineup)  # Fixed typo: 'current_Lineup' -> 'current_lineup'

    def get_team_stats(self, lineup):
        lineup_df = results_df[results_df['Athlete'].isin(lineup)]
        return {
            'Team_Offensive_Score': lineup_df['ORtg'].sum() / len(lineup) if len(lineup) > 0 else 0,
            'Team_Defensive_Score': lineup_df['DRtg'].sum() / len(lineup) if len(lineup) > 0 else 0,
            #'Team_Game_Score': lineup_df['Game Score'].sum() / len(lineup) if 'Game Score' in lineup_df.columns and len(lineup) > 0 else 0
        }

    def get_opp_stats(self):
        opp_df = results_df[results_df['Athlete'].isin(self.opp_lineup)]
        return {
            'Team_Offensive_Score_Opp': opp_df['ORtg'].sum() / len(self.opp_lineup) if len(self.opp_lineup) > 0 else 0,
            'Team_Defensive_Score_Opp': opp_df['DRtg'].sum() / len(self.opp_lineup) if len(self.opp_lineup) > 0 else 0,
           # 'Team_Game_Score_Opp': opp_df['Game Score'].sum() / len(self.opp_lineup) if 'Game Score' in opp_df.columns and len(self.opp_lineup) > 0 else 0
        }

    def step(self, action):
        player = self.team_players[action]
        if player in self.current_lineup:
            self.current_lineup.remove(player)  # Remove player
        elif len(self.current_lineup) < self.max_lineup_size:
            self.current_lineup.append(player)  # Add player

        # Calculate reward (win probability)
        if len(self.current_lineup) == self.max_lineup_size:
            team_stats = self.get_team_stats(self.current_lineup)
            opp_stats = self.get_opp_stats()
            features_dict = {**team_stats, **opp_stats}
            features_df = pd.DataFrame([features_dict], columns=self.feature_names)
            reward = self.win_predictor.predict_proba(features_df)[0][1]  # Probability of win
        else:
            reward = 0  # No reward until lineup is complete

        done = len(self.current_lineup) == self.max_lineup_size
        return tuple(self.current_lineup), reward, done

# Q-Learning Algorithm
def q_learning(team_players, opp_lineup, win_predictor, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    env = LineupEnv(team_players, opp_lineup, win_predictor)
    q_table = {}  # State-action value table

    for _ in range(episodes):
        state = env.reset()
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:  # Exploration
                action = random.randint(0, env.action_space - 1)
            else:  # Exploitation
                q_values = [q_table.get((state, a), 0) for a in range(env.action_space)]
                action = np.argmax(q_values)

            next_state, reward, done = env.step(action)

            # Update Q-table
            old_value = q_table.get((state, action), 0)
            next_max = max([q_table.get((next_state, a), 0) for a in range(env.action_space)], default=0)
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[(state, action)] = new_value

            state = next_state

    # Extract the best lineup
    state = env.reset()
    done = False
    while not done:
        q_values = [q_table.get((state, a), 0) for a in range(env.action_space)]
        action = np.argmax(q_values)
        state, reward, done = env.step(action)

    return env.current_lineup, reward

# Example Usage
team_players = results_df['Athlete'].unique().tolist()[:10]  # First 10 players as team roster
opp_lineup = ['Alyssa Geary', 'A\'Jah Davis', 'A\'Riana Gray', 'Player4', 'Player5']  # Example opposition lineup

# Train the win predictor model
win_predictor = train_win_predictor(df_filtered)

# Run Q-learning to find the best lineup
best_lineup, win_prob = q_learning(team_players, opp_lineup, win_predictor)
print(f"Best Lineup: {best_lineup}")
print(f"Predicted Win Probability: {win_prob:.4f}")

Logistic Regression Model Accuracy: 0.9242
Best Lineup: ['Aaliyah Gayles', 'Aaliyah Frazier', "A'Riana Gray", "A'Jah Davis", "A'Moni Waiters"]
Predicted Win Probability: 0.5376


In [ ]:
df_filtered.head()

,Team Display Name,Date,Opponet,PTS,MIN,FGM,FGA,PM,PA,FTM,...,AST,BLK,STL,TO,PF,Team Score,Win,Game Score,ORtg,DRtg
0,Providence Friars,2020-11-25,Quinnipiac Bobcats,17,38,7,17,0,2,3,...,4,3,1,4,5,65,0,9.7,557.204716,333.083903
1,Providence Friars,2020-11-25,Quinnipiac Bobcats,15,15,7,12,0,0,1,...,2,1,1,2,4,65,0,10.4,2250.060839,1746.778456
2,Providence Friars,2020-11-25,Quinnipiac Bobcats,5,38,1,4,0,0,3,...,0,0,2,1,2,65,0,6.3,97.330142,142.575923
3,Providence Friars,2020-11-25,Quinnipiac Bobcats,14,36,4,11,2,8,4,...,3,0,0,0,1,65,0,10.5,187.662777,160.759644
4,Providence Friars,2020-11-25,Quinnipiac Bobcats,8,26,2,7,0,0,4,...,3,0,0,2,3,65,0,4.1,381.582921,276.350154


In [ ]:
print(np.isinf(results_df[['ORtg', 'DRtg']]).sum())

ORtg    0
DRtg    0
dtype: int64


In [ ]:
results_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df.dropna(inplace=True)

In [ ]:
def get_stats(lineup, results_df, stat_cols=['ORtg', 'DRtg', 'Game Score']):
    lineup_df = results_df[results_df['Athlete'].isin(lineup)]
    stats = {f'Team_{col}': lineup_df[col].sum() / len(lineup) if len(lineup) > 0 and col in lineup_df.columns else 0
             for col in stat_cols}
    return stats

def greedy_lineup(team_players, opp_lineup, win_predictor, results_df):
    feature_names = ['Team_Offensive_Score', 'Team_Defensive_Score', 'Team_Game_Score',
                     'Team_Offensive_Score_Opp', 'Team_Defensive_Score_Opp', 'Team_Game_Score_Opp']
    opp_stats = get_stats(opp_lineup, results_df)
    lineup = []
    remaining_players = team_players.copy()

    for _ in range(5):
        best_player = None
        best_prob = -1
        for player in remaining_players:
            temp_lineup = lineup + [player]
            team_stats = get_stats(temp_lineup, results_df)
            features_dict = {**team_stats, **{f"{k}_Opp": v for k, v in opp_stats.items()}}
            features_df = pd.DataFrame([features_dict], columns=feature_names)
            prob = win_predictor.predict_proba(features_df)[0][1]
            if prob > best_prob:
                best_prob = prob
                best_player = player
        lineup.append(best_player)
        remaining_players.remove(best_player)

    return lineup, best_prob

# Example usage
best_lineup, win_prob = greedy_lineup(team_players, opp_lineup, win_predictor, results_df)
print(f"Best Lineup: {best_lineup}")
print(f"Predicted Win Probability: {win_prob:.4f}")

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
import random

def genetic_algorithm(team_players, opp_lineup, win_predictor, results_df, pop_size=50, generations=100):
    feature_names = ['Team_Offensive_Score', 'Team_Defensive_Score', 'Team_Game_Score',
                     'Team_Offensive_Score_Opp', 'Team_Defensive_Score_Opp', 'Team_Game_Score_Opp']
    opp_stats = get_stats(opp_lineup, results_df)

    # Initialize population
    population = [random.sample(team_players, 5) for _ in range(pop_size)]

    for _ in range(generations):
        # Evaluate fitness
        fitness = []
        for lineup in population:
            team_stats = get_stats(lineup, results_df)
            features_dict = {**team_stats, **{f"{k}_Opp": v for k, v in opp_stats.items()}}
            features_df = pd.DataFrame([features_dict], columns=feature_names)
            prob = win_predictor.predict_proba(features_df)[0][1]
            fitness.append(prob)

        # Selection (top 50%)
        sorted_pairs = sorted(zip(fitness, population), reverse=True)
        population = [pair[1] for pair in sorted_pairs[:pop_size//2]]

        # Crossover and mutation
        while len(population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child = list(set(parent1[:3] + parent2[2:]))  # Simple crossover
            if len(child) < 5:
                child += random.sample([p for p in team_players if p not in child], 5 - len(child))
            if random.random() < 0.1:  # Mutation (10% chance)
                child[random.randint(0, 4)] = random.choice([p for p in team_players if p not in child])
            population.append(child[:5])

    # Best lineup
    best_lineup = max(population, key=lambda x: win_predictor.predict_proba(
        pd.DataFrame([{**get_stats(x, results_df), **{f"{k}_Opp": v for k, v in opp_stats.items()}}], columns=feature_names))[0][1])
    best_prob = win_predictor.predict_proba(pd.DataFrame([{**get_stats(best_lineup, results_df), **{f"{k}_Opp": v for k, v in opp_stats.items()}}], columns=feature_names))[0][1]
    return best_lineup, best_prob

# Example usage
best_lineup, win_prob = genetic_algorithm(team_players, opp_lineup, win_predictor, results_df)
print(f"Best Lineup: {best_lineup}")
print(f"Predicted Win Probability: {win_prob:.4f}")

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values